In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
model_nm = 'distilbert-base-uncased'

In [3]:
model = AutoModelForMaskedLM.from_pretrained(model_nm)
tokenizer = AutoTokenizer.from_pretrained(model_nm)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
inputs = tokenizer('This is a great [MASK]', return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 2307,  103,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [5]:
token_logits = model(**inputs).logits

In [6]:
token_logits.shape

torch.Size([1, 7, 30522])

In [7]:
mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
mask_token_index

tensor([5])

In [8]:
mask_token_logits = token_logits[0, mask_token_index]
mask_token_logits 

tensor([[-7.1287, -7.0408, -7.4466,  ..., -6.2678, -7.3546, -2.8264]],
       grad_fn=<IndexBackward0>)

In [9]:
top_5_tokens = torch.topk(mask_token_logits, 10).indices[0].tolist()
top_5_tokens

[999, 1012, 3066, 6172, 1025, 1029, 3112, 6547, 2518, 3861]

In [10]:
for i in top_5_tokens:
    print(tokenizer.decode([i]))

!
.
deal
adventure
;
?
success
mystery
thing
picture


In [11]:
from datasets import load_dataset

In [12]:
imdb_dataset = load_dataset('imdb')
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [13]:
imdb_dataset['train'][0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [14]:
result = tokenizer(imdb_dataset['train'][0]['text'])
result

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

In [15]:
len(result['input_ids'])

363

In [16]:
len(result.word_ids())

363

In [17]:
tokenizer.is_fast

True

In [18]:
result.word_ids(0) == result.word_ids()

True

In [19]:
result['word_ids'] = result.word_ids()

In [20]:
result

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

In [21]:
def tokenize_func(item):
    result = tokenizer(item['text'])
    if tokenizer.is_fast:
        result['word_ids'] = [result.word_ids(i) for i in range(len(result.input_ids))]
    return result

In [22]:
tokenizer.model_max_length

512

In [23]:
tokenize_dataset = imdb_dataset.map(tokenize_func, batched = True, remove_columns=['text', 'label'])
tokenize_dataset

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [24]:
chunk_size = 128

In [25]:
samples = tokenize_dataset['train'][:3]
len(samples)

3

In [26]:
for idx, sample in enumerate(samples['input_ids']):
    print(idx, len(sample))

0 363
1 304
2 133


In [27]:
samples.keys()

dict_keys(['input_ids', 'attention_mask', 'word_ids'])

In [28]:
sum([[1, 2, 3, 4, 5], [6, 7, 8]], [])

[1, 2, 3, 4, 5, 6, 7, 8]

In [29]:
concatenate_example = {i: sum(samples[i], []) for i in samples.keys()}

In [30]:
length = len(concatenate_example['input_ids'])
length

800

In [31]:
len(concatenate_example)

3

In [32]:
list(range(0, 10, 2))

[0, 2, 4, 6, 8]

In [33]:
chunk = {
    key: [item[i: i + chunk_size] for i in range(0, length, chunk_size)] 
    for key, item in concatenate_example.items()
}

In [34]:
len(chunk['input_ids'])

7

In [35]:
for i in chunk['input_ids']:
    print(len(i))

128
128
128
128
128
128
32


In [36]:
(length // chunk_size) * chunk_size

768

In [37]:
def group_texts(examples):
    concatenate_examples = {i: sum(examples[i], []) for i in examples.keys()}
    total_length = len(concatenate_examples['input_ids'])
    total_length = (total_length // chunk_size) * chunk_size
    chunks = {key: [item[i: i + chunk_size] for i in range(0, total_length, chunk_size)] for key, item in concatenate_examples.items()}
    chunks['labels'] = chunks['input_ids'].copy()
    return chunks

In [38]:
lm_datasets = tokenize_dataset.map(group_texts, batched = True)
lm_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [39]:
import collections
import numpy as np
from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [40]:
samples = [lm_datasets['train'][i] for i in range(2)]
len(samples)

batch = whole_word_masking_data_collator(samples)

for i in batch['input_ids']:
    print(tokenizer.decode(i))

[CLS] i [MASK] i am [MASK] - yellow from my video [MASK] because of [MASK] the controversy that surrounded [MASK] when [MASK] was first released in 1967. i also heard that at [MASK] it was [MASK] by u [MASK] s. [MASK] if it [MASK] tried to enter this [MASK], therefore being [MASK] fan of films considered " controversial " i [MASK] had to see [MASK] for myself. < br / > < br / > [MASK] plot is centered around a young swedish drama student named lena who wants to [MASK] [MASK] she [MASK] about life. in particular she wants to focus her attentions to making some [MASK] of documentary on what the average swede thought about certain political issues [MASK]
as the vietnam [MASK] and race issues in the united states [MASK] in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and [MASK] men. < br / > < br / > what kills me about i am curious - [MASK] is that 40 years ago, this was considered pornograp

In [41]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [42]:
train_size = 10000
test_size = int(0.2 * train_size)
train_size, test_size

(10000, 2000)

In [43]:
downsampled_dataset = lm_datasets['train'].train_test_split(train_size=train_size, test_size = test_size, seed = 42)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2000
    })
})

In [44]:
from transformers import TrainingArguments, Trainer

In [132]:
bs = 64
logging_steps = len(downsampled_dataset['train']) // bs
args = TrainingArguments(
    output_dir=f"{model_nm}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    fp16=True,
    logging_steps=logging_steps,
    remove_unused_columns = False,
    report_to = 'none',
    push_to_hub = True,
    num_train_epochs=10
)

In [46]:
from transformers import DataCollatorForWholeWordMask

In [47]:
data_collator = DataCollatorForWholeWordMask(tokenizer = tokenizer, mlm_probability = 0.15)

In [48]:
test = data_collator([downsampled_dataset['train'][0]['input_ids']])
test

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


{'input_ids': tensor([[ 1010,   103,   103,  2035,  2017,  2963,  2005,  2019,  3178,   103,
           1037,  2431,  2003,   103,  2152,  1011,  8219,  1059, 14014,  2183,
           2006,  2055,  2129,  2010,  4331,  2024,  2190,  1010,  2030,  2054,
          21864, 15952,  3538,   103,  2759,  3226,  2003, 16356,  2989,  2010,
          11281,  2012,   103,  2617,  1012,  2002,  2036,  4858,  2051,   103,
          22889,  8490,  2125,  3152,  2008, 13886,  2987,  1005,   103,  2066,
           1010,  2242,   103,  2245,  2001,  9235,  2005, 23160,  2066,  2033,
           1012,  7543,  2108,  1999,  1037,  2597,  2066,  2010,  2017,  1005,
           1040,  2228,   103,  1005,  1040,  3046,   103,  2191,  1037,  2391,
           2055,  5988,  1037,   103,  2062,  9414,  2135,  2084,  2023,  1012,
           2011,  2437,   103,  5372,  2143,  3383,  1010,  2028,  2007,  2070,
           4784,  1998,  1037, 11519,  3252,   103,  2030,   103,  2028,  2008,
           3475,  1005,  10

In [49]:
tokenizer.decode(test['input_ids'][0])

", [MASK] [MASK] all you hear for an hour [MASK] a half is [MASK] high - pitched whine going on about how his politics are best, or what quirky piece [MASK] popular culture is tickling his fancy at [MASK] moment. he also finds time [MASK] slag off films that clicked doesn'[MASK] like, something [MASK] thought was reserved for losers like me. surely being in a position like his you'd think [MASK]'d try [MASK] make a point about cinema a [MASK] more intelligently than this. by making [MASK] proper film perhaps, one with some ideas and a decent structure [MASK] or [MASK] one that isn't 71 dominated [MASK] [MASK] annoying"

In [74]:
tokenizer.decode([2061, 2008])

'so that'

In [77]:
from huggingface_hub import notebook_login

notebook_login()

In [133]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset= downsampled_dataset['train'],
    eval_dataset = downsampled_dataset['test'],
    data_collator= data_collator,
    tokenizer = tokenizer,
)

/kaggle/working/distilbert-base-uncased-finetuned-imdb is already a clone of https://huggingface.co/Sonali-Behera/distilbert-base-uncased-finetuned-imdb. Make sure you pull the latest changes with `repo.git_pull()`.


In [134]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


{'eval_loss': 2.682955026626587,
 'eval_runtime': 5.7538,
 'eval_samples_per_second': 347.593,
 'eval_steps_per_second': 5.561}

In [135]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.554500,2.685557
2,2.611800,2.656390
3,2.660100,2.658659
4,2.675400,2.616541
5,2.658600,2.614622
6,2.632900,2.621978
7,2.628200,2.596710
8,2.628100,2.642364
9,2.607400,2.614133
10,2.606100,2.597775


/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


TrainOutput(global_step=1570, training_loss=2.6252612788206453, metrics={'train_runtime': 1045.6802, 'train_samples_per_second': 95.632, 'train_steps_per_second': 1.501, 'total_flos': 3314028902400000.0, 'train_loss': 2.6252612788206453, 'epoch': 10.0})

In [136]:
trainer.evaluate()

{'eval_loss': 2.6268422603607178,
 'eval_runtime': 5.7521,
 'eval_samples_per_second': 347.697,
 'eval_steps_per_second': 5.563,
 'epoch': 10.0}

In [55]:
from transformers import pipeline

In [56]:
len(downsampled_dataset['test'][0])

4

In [57]:
test1 = tokenizer('This is a great [MASK]')
test1['word_ids'] = test1.word_ids()
test1

{'input_ids': [101, 2023, 2003, 1037, 2307, 103, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'word_ids': [None, 0, 1, 2, 3, 4, None]}

In [67]:
pred = trainer.predict([downsampled_dataset['test'][0]])
pred

PredictionOutput(predictions=array([[[-10.2109375, -10.140625 , -10.2109375, ...,  -9.546875 ,
          -9.7421875,  -7.171875 ],
        [ -9.3671875,  -9.359375 ,  -9.3671875, ...,  -7.828125 ,
          -7.65625  ,  -7.7226562],
        [-13.3671875, -13.4375   , -13.328125 , ..., -11.0546875,
         -11.171875 , -10.5703125],
        ...,
        [-11.3046875, -11.015625 , -11.078125 , ..., -11.       ,
         -10.5546875, -10.46875  ],
        [ -9.9375   ,  -9.8515625,  -9.9375   , ...,  -9.421875 ,
          -8.953125 ,  -9.234375 ],
        [ -8.9140625,  -8.6953125,  -8.78125  , ...,  -8.59375  ,
          -8.59375  ,  -7.2929688]]], dtype=float32), label_ids=array([[1012, 2138, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, 2000, -100, 2004, -100, -100, -100, -100, -100,
        -100, 2231, -100, -100, -100, 2064, -100, -100, -100, -100, -100,
        -100, -100, -1

In [137]:
trainer.push_to_hub(commit_message='push trainer with 10 epochs')

Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

To https://huggingface.co/Sonali-Behera/distilbert-base-uncased-finetuned-imdb
   f5e09e8..843d635  main -> main

To https://huggingface.co/Sonali-Behera/distilbert-base-uncased-finetuned-imdb
   843d635..4e7723e  main -> main



'https://huggingface.co/Sonali-Behera/distilbert-base-uncased-finetuned-imdb/commit/843d6359712d89a7ed975844a9392d53c68c47cb'

In [68]:
tokenizer.decode(downsampled_dataset['test'][0]['labels'])

'. because you got to see them as real characters this makes you like them more as an audience, and makes you more sympathetic to them as totally the victims of the white government, who you can not sympathise with. the singing of the students is correct because we know from accounts that the students in the riot were singing and dancing before it became violent. the clothing of the students in sarafina is very similar to the clothing shown in photos from soweto. they made the movie actually in soweto, which is why it looks very accurate in many parts. all these things make the film more accurate for someone using'

In [69]:
tokenizer.decode(lm_datasets['train'][10]['input_ids'])

'scrape to find value in its boring pseudo revolutionary political spewings.. but if it weren\'t for the censorship scandal, it would have been ignored, then forgotten. < br / > < br / > instead, the " i am blank, blank " rhythymed title was repeated endlessly for years as a titilation for porno films ( i am curious, lavender - for gay films, i am curious, black - for blaxploitation films, etc.. ) and every ten years or so the thing rises from the dead, to be viewed by a new generation of suckers who want'

In [61]:
tokenizer.decode(downsampled_dataset['train'][1]['input_ids'])

'started thinking this was worthy of a 7, but as the film went on it dropped rapidly to a 4, then earning a 3 after the silliness of the wedding scene. this was about as cold and sterile a movie as i have seen. a terrible waste of a good story. [SEP] [CLS] one thing that astonished me about this film ( and not in a good way ) was that nathan stoltzfus, who seems to pride himself on being the major historian on the topic of the rosenstrasse, was one of the historians working on this film, considering how much of the actual events were altered or disregarded. < br'

In [71]:
pred.predictions[0]

array([[-10.2109375, -10.140625 , -10.2109375, ...,  -9.546875 ,
         -9.7421875,  -7.171875 ],
       [ -9.3671875,  -9.359375 ,  -9.3671875, ...,  -7.828125 ,
         -7.65625  ,  -7.7226562],
       [-13.3671875, -13.4375   , -13.328125 , ..., -11.0546875,
        -11.171875 , -10.5703125],
       ...,
       [-11.3046875, -11.015625 , -11.078125 , ..., -11.       ,
        -10.5546875, -10.46875  ],
       [ -9.9375   ,  -9.8515625,  -9.9375   , ...,  -9.421875 ,
         -8.953125 ,  -9.234375 ],
       [ -8.9140625,  -8.6953125,  -8.78125  , ...,  -8.59375  ,
         -8.59375  ,  -7.2929688]], dtype=float32)

In [80]:
from huggingface_hub import get_full_repo_name

In [82]:
model_name = 'distilbert-base-uncased-finetuned-imdb'
repo_name = get_full_repo_name(model_name)
repo_name

'Sonali-Behera/distilbert-base-uncased-finetuned-imdb'

In [84]:
from huggingface_hub import Repository

In [86]:
output_dir = model_name
repo = Repository(output_dir, clone_from = repo_name)

/kaggle/working/distilbert-base-uncased-finetuned-imdb is already a clone of https://huggingface.co/Sonali-Behera/distilbert-base-uncased-finetuned-imdb. Make sure you pull the latest changes with `repo.git_pull()`.


In [87]:
from accelerate import Accelerator

In [92]:
model

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [88]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [89]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, downsampled_dataset['train'], downsampled_dataset['test']
)

In [91]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
if accelerator.is_main_process:
    tokenizer.save_pretrained(output_dir)
    repo.push_to_hub(
        commit_message='Testing', blocking=False
    )

In [94]:
masking = pipeline('fill-mask', model = 'Sonali-Behera/distilbert-base-uncased-finetuned-imdb')

In [96]:
masking('This is a great [MASK]')

[{'score': 0.4412679672241211,
  'token': 999,
  'token_str': '!',
  'sequence': 'this is a great!'},
 {'score': 0.2978465259075165,
  'token': 1012,
  'token_str': '.',
  'sequence': 'this is a great.'},
 {'score': 0.03835844621062279,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'this is a great film'},
 {'score': 0.028337379917502403,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'this is a great movie'},
 {'score': 0.010450121946632862,
  'token': 3066,
  'token_str': 'deal',
  'sequence': 'this is a great deal'}]

In [97]:
masking('I should buy a [MASK]')

[{'score': 0.3545718193054199,
  'token': 1012,
  'token_str': '.',
  'sequence': 'i should buy a.'},
 {'score': 0.049423087388277054,
  'token': 999,
  'token_str': '!',
  'sequence': 'i should buy a!'},
 {'score': 0.04307328909635544,
  'token': 4966,
  'token_str': 'dvd',
  'sequence': 'i should buy a dvd'},
 {'score': 0.033906590193510056,
  'token': 6100,
  'token_str': 'copy',
  'sequence': 'i should buy a copy'},
 {'score': 0.030957302078604698,
  'token': 7281,
  'token_str': 'ticket',
  'sequence': 'i should buy a ticket'}]

In [98]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [108]:
samples = [downsampled_dataset['test'][i] for i in range(2)]

In [109]:
samples.map(insert_random_mask)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 samples.map(insert_random_mask)                                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'list' object has no attribute 'map'

In [116]:
tokenizer.decode(downsampled_dataset['test'][0]['input_ids'])

'. because you got to see them as real characters this makes you like them more as an audience, and makes you more sympathetic to them as totally the victims of the white government, who you can not sympathise with. the singing of the students is correct because we know from accounts that the students in the riot were singing and dancing before it became violent. the clothing of the students in sarafina is very similar to the clothing shown in photos from soweto. they made the movie actually in soweto, which is why it looks very accurate in many parts. all these things make the film more accurate for someone using'

In [117]:
masking('ecause you got to see them as real characters this makes you like them more as an [MASK]')

[{'score': 0.25816231966018677,
  'token': 1012,
  'token_str': '.',
  'sequence': 'ecause you got to see them as real characters this makes you like them more as an.'},
 {'score': 0.14795410633087158,
  'token': 3364,
  'token_str': 'actor',
  'sequence': 'ecause you got to see them as real characters this makes you like them more as an actor'},
 {'score': 0.0428636372089386,
  'token': 3883,
  'token_str': 'actress',
  'sequence': 'ecause you got to see them as real characters this makes you like them more as an actress'},
 {'score': 0.030408352613449097,
  'token': 21751,
  'token_str': 'entertainer',
  'sequence': 'ecause you got to see them as real characters this makes you like them more as an entertainer'},
 {'score': 0.027762606739997864,
  'token': 3265,
  'token_str': 'individual',
  'sequence': 'ecause you got to see them as real characters this makes you like them more as an individual'}]

In [118]:
masking('they made the movie actually in [MASK]')

[{'score': 0.44295230507850647,
  'token': 1012,
  'token_str': '.',
  'sequence': 'they made the movie actually in.'},
 {'score': 0.032563161104917526,
  'token': 1025,
  'token_str': ';',
  'sequence': 'they made the movie actually in ;'},
 {'score': 0.024662215262651443,
  'token': 1029,
  'token_str': '?',
  'sequence': 'they made the movie actually in?'},
 {'score': 0.023442735895514488,
  'token': 999,
  'token_str': '!',
  'sequence': 'they made the movie actually in!'},
 {'score': 0.008598772808909416,
  'token': 2605,
  'token_str': 'france',
  'sequence': 'they made the movie actually in france'}]

In [120]:
masking('the singing of the students is correct [MASK] we know from accounts that')

[{'score': 0.33871564269065857,
  'token': 1012,
  'token_str': '.',
  'sequence': 'the singing of the students is correct. we know from accounts that'},
 {'score': 0.18737328052520752,
  'token': 1998,
  'token_str': 'and',
  'sequence': 'the singing of the students is correct and we know from accounts that'},
 {'score': 0.09713467210531235,
  'token': 1010,
  'token_str': ',',
  'sequence': 'the singing of the students is correct, we know from accounts that'},
 {'score': 0.07213137298822403,
  'token': 2021,
  'token_str': 'but',
  'sequence': 'the singing of the students is correct but we know from accounts that'},
 {'score': 0.06839676201343536,
  'token': 2004,
  'token_str': 'as',
  'sequence': 'the singing of the students is correct as we know from accounts that'}]

In [121]:
tokenizer.decode(downsampled_dataset['test'][45]['input_ids'])

'concept is not without merit. < br / > < br / > we are left wondering why a loving couple - a father and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining. this loss of presence in the real world is, rather too obviously and unnecessarily, contrasted with the son having enlisted in the armed forces. why not the circus, so we can at least appreciate some colour? we are left with a gnawing sense of loss, but sadly no enlightenment, which is bewildering given the film is apparently about some'

In [122]:
masking('we are left with a [MASK] sense of loss, but sadly no enlightenment')

[{'score': 0.15367326140403748,
  'token': 13769,
  'token_str': 'profound',
  'sequence': 'we are left with a profound sense of loss, but sadly no enlightenment'},
 {'score': 0.10520612448453903,
  'token': 2784,
  'token_str': 'deep',
  'sequence': 'we are left with a deep sense of loss, but sadly no enlightenment'},
 {'score': 0.05755991488695145,
  'token': 2307,
  'token_str': 'great',
  'sequence': 'we are left with a great sense of loss, but sadly no enlightenment'},
 {'score': 0.04250901937484741,
  'token': 2844,
  'token_str': 'strong',
  'sequence': 'we are left with a strong sense of loss, but sadly no enlightenment'},
 {'score': 0.041556466370821,
  'token': 3056,
  'token_str': 'certain',
  'sequence': 'we are left with a certain sense of loss, but sadly no enlightenment'}]

In [123]:
masking('a [MASK] and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining')

[{'score': 0.4670730531215668,
  'token': 2388,
  'token_str': 'mother',
  'sequence': 'a mother and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining'},
 {'score': 0.2596692144870758,
  'token': 2269,
  'token_str': 'father',
  'sequence': 'a father and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining'},
 {'score': 0.16834187507629395,
  'token': 2564,
  'token_str': 'wife',
  'sequence': 'a wife and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining'},
 {'score': 0.029327429831027985,
  'token': 3129,
  'token_str': 'husband',
  'sequence': 'a husband and son no less - should be so estranged from the real world that their own world is preferable when claustrophobic beyond all imagining'},
 {'score': 0.02290317229926586,
  'token': 2684,
  

In [124]:
tokenizer.decode(downsampled_dataset['test'][98]['input_ids'])

'that 50 % of a good film is the music and though i\'m not certain i think the title theme was a simple but moving clarinet solo of " what a friend we have in jesus ". the film then went on to disprove that! am i right or wrong? [SEP] [CLS] i saw this 25 years ago on pbs. it was very difficult to watch. so real. to watch this small family struggle in the winter was heart rending. no time for courting : fate has thrown us together and we put our shoulders to the grindstone and make it work. this was based on the woman\'s actual diary,'

In [125]:
masking('i saw this [MASK] years ago on pbs')

[{'score': 0.11962835490703583,
  'token': 2184,
  'token_str': '10',
  'sequence': 'i saw this 10 years ago on pbs'},
 {'score': 0.06810285896062851,
  'token': 2048,
  'token_str': 'two',
  'sequence': 'i saw this two years ago on pbs'},
 {'score': 0.06252306699752808,
  'token': 2322,
  'token_str': '20',
  'sequence': 'i saw this 20 years ago on pbs'},
 {'score': 0.048727065324783325,
  'token': 1016,
  'token_str': '2',
  'sequence': 'i saw this 2 years ago on pbs'},
 {'score': 0.03928649052977562,
  'token': 2382,
  'token_str': '30',
  'sequence': 'i saw this 30 years ago on pbs'}]

In [127]:
masking('this was based on the [MASK]\'s actual diary')

[{'score': 0.1298181265592575,
  'token': 3166,
  'token_str': 'author',
  'sequence': "this was based on the author's actual diary"},
 {'score': 0.11708644777536392,
  'token': 2143,
  'token_str': 'film',
  'sequence': "this was based on the film's actual diary"},
 {'score': 0.05739942938089371,
  'token': 2839,
  'token_str': 'character',
  'sequence': "this was based on the character's actual diary"},
 {'score': 0.03526931628584862,
  'token': 3213,
  'token_str': 'writer',
  'sequence': "this was based on the writer's actual diary"},
 {'score': 0.02874480001628399,
  'token': 3185,
  'token_str': 'movie',
  'sequence': "this was based on the movie's actual diary"}]

In [129]:
masking('what is your [MASK]')

[{'score': 0.8263999819755554,
  'token': 1029,
  'token_str': '?',
  'sequence': 'what is your?'},
 {'score': 0.03628050908446312,
  'token': 1012,
  'token_str': '.',
  'sequence': 'what is your.'},
 {'score': 0.017996083945035934,
  'token': 2171,
  'token_str': 'name',
  'sequence': 'what is your name'},
 {'score': 0.014039468951523304,
  'token': 999,
  'token_str': '!',
  'sequence': 'what is your!'},
 {'score': 0.004874095320701599,
  'token': 2166,
  'token_str': 'life',
  'sequence': 'what is your life'}]

In [130]:
masking('the movie is full of [MASK]')

[{'score': 0.46089789271354675,
  'token': 1012,
  'token_str': '.',
  'sequence': 'the movie is full of.'},
 {'score': 0.04534144699573517,
  'token': 20096,
  'token_str': 'surprises',
  'sequence': 'the movie is full of surprises'},
 {'score': 0.02637871541082859,
  'token': 4569,
  'token_str': 'fun',
  'sequence': 'the movie is full of fun'},
 {'score': 0.02590741403400898,
  'token': 999,
  'token_str': '!',
  'sequence': 'the movie is full of!'},
 {'score': 0.011840630322694778,
  'token': 4933,
  'token_str': 'stuff',
  'sequence': 'the movie is full of stuff'}]